Let’s go through some of the most popular features of Gradio! Here are Gradio’s key features:

Adding example inputs

Passing custom error messages

Adding descriptive content

Setting up flagging

Preprocessing and postprocessing

Styling demos

Queuing users

Iterative outputs

Progress bars

Batch functions

Running on collaborative notebooks

In [ ]:
pip install gradio

Example Inputs

In [ ]:
import gradio as gr

def calculator(num1, operation, num2):
    if operation == "add":
        return num1 + num2
    elif operation == "subtract":
        return num1 - num2
    elif operation == "multiply":
        return num1 * num2
    elif operation == "divide":
        if num2 == 0:
            raise gr.Error("Cannot divide by zero!")
        return num1 / num2

demo = gr.Interface(
    calculator,
    [
        "number",
        gr.Radio(["add", "subtract", "multiply", "divide"]),
        "number"
    ],
    "number",
    examples=[
        [5, "add", 3],
        [4, "divide", 2],
        [-4, "multiply", 2.5],
        [0, "subtract", 1.2],
    ],
    title="Toy Calculator",
    description="Here's a sample toy calculator. Allows you to calculate things like $2+2=4$",
)
demo.launch()

Alerts

In [ ]:
import gradio as gr
def start_process(name, success):
    gr.Info("Starting process")
    if name is None:
        gr.Warning("Name is empty")
    if success == False:
        raise gr.Error("Process failed")

Descriptive Content

There are three arguments in the Interface constructor to specify where this content should go:

title: which accepts text and can display it at the very top of interface, and also becomes the page title.

description: which accepts text, markdown or HTML and places it right under the title.

article: which also accepts text, markdown or HTML and places it below the interface.


https://github.com/gradio-app/gradio/blob/main/guides/assets/annotated.png?raw=true




In [ ]:
gr.Number(label='Age', info='In years, must be greater than 0')

# **Flagging**

By default, an Interface will have “Flag” button. When a user testing your Interface sees input with interesting output, such as erroneous or unexpected model behaviour, they can flag the input for you to review. Within the directory provided by the flagging_dir= argument to the Interface constructor, a CSV file will log the flagged inputs. If the interface involves file data, such as for Image and Audio components, folders will be created to store those flagged data as well.

+-- sepia.py

+-- flagged/

|   +-- logs.csv

|   +-- im/

|   |   +-- 0.png

|   |   +-- 1.png

|   +-- Output/

|   |   +-- 0.png

|   |   +-- 1.png


# **Preprocessing and Postprocessing**

When a component is used as an input, Gradio automatically handles the preprocessing needed to convert the data from a type sent by the user’s browser (such as a base64 representation of a webcam snapshot) to a form that can be accepted by your function (such as a numpy array).

Similarly, when a component is used as an output, Gradio automatically handles the postprocessing needed to convert the data from what is returned by your function (such as a list of image paths) to a form that can be displayed in the user’s browser (such as a Gallery of images in base64 format).

In [ ]:
img = gr.Image(shape=(100, 100), type="pil")

img = gr.Image(invert_colors=True, type="numpy")


# **Styling**

Gradio themes are the easiest way to customize the look and feel of your app. You can choose from a variety of themes, or create your own. To do so, pass the theme= kwarg to the Interface constructor. For example:

In [ ]:
demo = gr.Interface(..., theme=gr.themes.Monochrome())

Gradio comes with a set of prebuilt themes which you can load from gr.themes.*. You can extend these themes or create your own themes from scratch - see the Theming guide for more details.

For additional styling ability, you can pass any CSS to your app using the css= kwarg. The base class for the Gradio app is gradio-container, so here’s an example that changes the background color of the Gradio app:

In [ ]:
with gr.Interface(css=".gradio-container {background-color: red}") as demo:

Some components can be additionally styled through the style() method. For example:



In [ ]:
img = gr.Image("lion.jpg").style(height='24', rounded=False)

# **Queuing**
If your app expects heavy traffic, use the queue() method to control processing rate. This will queue up calls so only a certain number of requests are processed at a single time. Queueing uses websockets, which also prevent network timeouts, so you should use queueing if the inference time of your function is long (> 1min).

With Interface:

In [ ]:
demo = gr.Interface(...).queue()
demo.launch()

With Blocks:

In [ ]:
with gr.Blocks() as demo:
    #...
demo.queue()
demo.launch()

You can control the number of requests processed at a single time as such:

In [ ]:
demo.queue(concurrency_count=3)

To specify only certain functions for queueing in Blocks:

In [ ]:
with gr.Blocks() as demo2:
    num1 = gr.Number()
    num2 = gr.Number()
    output = gr.Number()
    gr.Button("Add").click(
        lambda a, b: a + b, [num1, num2], output)
    gr.Button("Multiply").click(
        lambda a, b: a * b, [num1, num2], output, queue=True)
demo2.launch()

# **Iterative Outputs**

In some cases, you may want to stream a sequence of outputs rather than show a single output at once. For example, you might have an image generation model and you want to show the image that is generated at each step, leading up to the final image. Or you might have a chatbot which streams its response one word at a time instead of returning it all at once.

In such cases, you can supply a generator function into Gradio instead of a regular function. Creating generators in Python is very simple: instead of a single return value, a function should yield a series of values instead. Usually the yield statement is put in some kind of loop. Here’s an example of an generator that simply counts up to a given number:

In [ ]:
def my_generator(x):
    for i in range(x):
        yield i

In [ ]:
pip install gradio


In [ ]:
import gradio as gr
import numpy as np
import time

# define core fn, which returns a generator {steps} times before returning the image
def fake_diffusion(steps):
    for _ in range(steps):
        time.sleep(1)
        image = np.random.random((600, 600, 3))
        yield image
    image = "https://gradio-builds.s3.amazonaws.com/diffusion_image/cute_dog.jpg"
    yield image


demo = gr.Interface(fake_diffusion, inputs=gr.Slider(1, 10, 3), outputs="image")

# define queue - required for generators
demo.queue()

demo.launch()

# **Progress Bars**

Gradio supports the ability to create a custom Progress Bars so that you have customizability and control over the progress update that you show to the user. In order to enable this, simply add an argument to your method that has a default value of a gr.Progress instance. Then you can update the progress levels by calling this instance directly with a float between 0 and 1, or using the tqdm() method of the Progress instance to track progress over an iterable, as shown below. Queueing must be enabled for progress updates.

In [ ]:
import gradio as gr
import time

def slowly_reverse(word, progress=gr.Progress()):
    progress(0, desc="Starting")
    time.sleep(1)
    progress(0.05)
    new_string = ""
    for letter in progress.tqdm(word, desc="Reversing"):
        time.sleep(0.25)
        new_string = letter + new_string
    return new_string

demo = gr.Interface(slowly_reverse, gr.Text(), gr.Text())

if __name__ == "__main__":
    demo.queue(concurrency_count=10).launch()

# **Batch Functions**

Gradio supports the ability to pass batch functions. Batch functions are just functions which take in a list of inputs and return a list of predictions.

For example, here is a batched function that takes in two lists of inputs (a list of words and a list of ints), and returns a list of trimmed words as output:

In [ ]:
import time

def trim_words(words, lens):
    trimmed_words = []
    time.sleep(5)
    for w, l in zip(words, lens):
        trimmed_words.append(w[:int(l)])
    return [trimmed_words]

The advantage of using batched functions is that if you enable queuing, the Gradio server can automatically batch incoming requests and process them in parallel, potentially speeding up your demo. Here’s what the Gradio code looks like (notice the batch=True and max_batch_size=16 — both of these parameters can be passed into event triggers or into the Interface class)

With Interface:

In [ ]:
demo = gr.Interface(trim_words, ["textbox", "number"], ["output"],
                    batch=True, max_batch_size=16)
demo.queue()
demo.launch()

With Blocks:

In [ ]:
import gradio as gr
import time

def trim_words(words, lens):
    trimmed_words = []
    time.sleep(5)
    for w, l in zip(words, lens):
        trimmed_words.append(w[:int(l)])
    return [trimmed_words]

with gr.Blocks() as demo:
    with gr.Row():
        word = gr.Textbox(label="word")
        leng = gr.Number(label="leng")
        output = gr.Textbox(label="Output")
    with gr.Row():
        run = gr.Button()

    event = run.click(trim_words, [word, leng], output, batch=True, max_batch_size=16)

demo.queue()
demo.launch()